In [1]:
from scipy import stats
from numpy import log, exp, sqrt

In [3]:
def call_option_price(S, E, T, rf, sigma):
    # calculate d1 and d2 params
    # t=0 => T-t=T
    d1 = (log(S/E)+(rf+sigma*sigma/2)*T)/(sigma*sqrt(T))
    d2 = d1-sigma*sqrt(T)

    return S*stats.norm.cdf(d1)-E*exp(-rf*T)*stats.norm.cdf(d2)

In [4]:
def put_option_price(S, E, T, rf, sigma):
    # calculate d1 and d2 params
    # t=0 => T-t=T
    d1 = (log(S/E)+(rf+sigma*sigma/2)*T)/(sigma*sqrt(T))
    d2 = d1-sigma*sqrt(T)

    return -S*stats.norm.cdf(-d1)+E*exp(-rf*T)*stats.norm.cdf(-d2)

In [6]:
def main():
    S0 = 100
    E = 100
    T = 1
    rf = 0.05
    sigma = 0.2

    call = call_option_price(S0, E, T, rf, sigma)
    print(call)

    put = put_option_price(S0, E, T, rf, sigma)
    print(put)

In [7]:
if __name__ == "__main__":
    main()

10.450583572185565
5.573526022256971
